# Test Diff

In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
from deepdiff import DeepDiff, parse_path
from pprint import pprint
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
import json

# Load Context Memory

In [2]:

from Utilities.local_make_general import invoke_context_get_block, invoke_context_save_block
# 1. Load the Context
Type_Refinery_Context = invoke_context_get_block()
#
# 2. Setup the  TR User Context
#
local = Type_Refinery_Context["local"]
me = local["me"]
team = local["team"]
company = local["company"]
systems = local["systems"]
assets = local["assets"]
#
# 3. Setup the Incident Context
#
incident = Type_Refinery_Context["incident"]
bundle_list = incident["bundle_list"]
sequence_start_objs = incident["sequence_start_objs"]
sequence_objs = incident["sequence_objs"]
task_objs = incident["task_objs"]
event_objs = incident["event_objs"]
impact_objs = incident["impact_objs"]
other_object_objs = incident["other_object_objs"]
incident_obj = incident["incident_obj"]
#
# 4. Load the TypeDB Context
#
typedb = Type_Refinery_Context["typedb"]
t_bundle_list = typedb["bundle_list"]
t_sequence_start_objs = typedb["sequence_start_objs"]
t_sequence_objs = typedb["sequence_objs"]
t_task_objs = typedb["task_objs"]
t_event_objs = typedb["event_objs"]
t_impact_objs = typedb["impact_objs"]
t_other_object_objs = typedb["other_object_objs"]
t_incident_obj = typedb["incident_obj"]

# Lets run some tests

In [3]:
print((incident_obj))
print("")
print(t_incident_obj)

{'type': 'incident', 'spec_version': '2.1', 'id': 'incident--415c986d-f2b3-4c88-abb4-58531b3c3a93', 'created': '2023-12-31T08:03:04.46545Z', 'modified': '2023-12-31T08:03:04.46545Z', 'name': 'potential phishing', 'extensions': {'extension-definition--ef765651-680c-498d-9894-99799f2fa126': {'extension_type': 'property-extension', 'investigation_status': 'new', 'incident_types': ['dissemination-phishing-emails'], 'sequence_start_refs': ['sequence--fc108d99-8262-4082-81dd-02411203d291', 'sequence--ac6f3beb-e28a-49ef-b0e4-2ddcd54fa2a2'], 'sequence_refs': ['sequence--b75a90bb-5a55-48c3-bdae-23c9e693a362', 'sequence--b75a90bb-5a55-48c3-bdae-23c9e693a362', 'sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be'], 'task_refs': ['task--8a676b0b-a835-4e13-8912-b912c3d9be72', 'task--3d8b10a7-0217-4784-a17d-26f55a68da0a'], 'event_refs': ['event--59026c8c-b8f7-4cbf-9740-a3b2b22bad7f'], 'other_object_refs': ['email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'user-account--597ad4d4-35ba-585d-8f6d-134a7503

In [4]:
diff = DeepDiff(t_incident_obj, incident_obj)
print(diff)

{'dictionary_item_added': [root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['impact_refs']], 'iterable_item_added': {"root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['sequence_refs'][2]": 'sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be', "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['task_refs'][1]": 'task--3d8b10a7-0217-4784-a17d-26f55a68da0a', "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][11]": 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][12]": 'observed-data--2a710640-d225-452f-a454-f110fa5c4c9e', "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][13]": 'sighting--fd535c1c-a12b-4e12-8d62-e99c752b8cdf', "root['extensions']['extension-definition--ef765651-680c-498d-9894-99

In [5]:
incident_obj["name"]= "Potential phishing"
del incident_obj["extensions"]["extension-definition--ef765651-680c-498d-9894-99799f2fa126"]["investigation_status"]
diff = DeepDiff(t_incident_obj, incident_obj, verbose_level=2)
pprint(diff, depth=4, indent=2)

{ 'dictionary_item_added': { "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['impact_refs']": [ 'impact--dfb745f7-a931-4f3a-a6ef-6e9f8ae4a018']},
  'dictionary_item_removed': { "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['investigation_status']": 'new'},
  'iterable_item_added': { "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][11]": 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f',
                           "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][12]": 'observed-data--2a710640-d225-452f-a454-f110fa5c4c9e',
                           "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['other_object_refs'][13]": 'sighting--fd535c1c-a12b-4e12-8d62-e99c752b8cdf',
                           "root['extensions']['extension-definition--ef765651-680c-498d-9894-99799f2fa126']['

In [7]:
for d, i in diff.items():
    print(f"------  {d}")
    if d == "dictionary_item_added":
        print("..... ITs an add statement")
        for k, v in i.items():
            test1 = parse_path(k, include_actions=True)
            test2 = parse_path(k)
            print(f"======= {test2}")
            #print(f",,, {test1}")
            print(v)
    if d == "dictionary_item_removed":
        print("..... ITs a remove statement")
        for k, v in i.items():
            test1 = parse_path(k, include_actions=True)
            test2 = parse_path(k)
            print(f"======= {test2}")
            #print(f",,, {test1}")
            print(v)
    if d == "values_changed":
        print("..... ITs an value changed statement")
        for k, v in i.items():
            test1 = parse_path(k, include_actions=True)
            test2 = parse_path(k)
            print(f"======= {test2}")
            #print(f",,, {test1}")
            print(v)
    if d == "iterable_item_added":
        print("..... ITs an iterable item added statement")
        for k, v in i.items():
            test1 = parse_path(k, include_actions=True)
            test2 = parse_path(k)
            print(f"======= {test2}")
            #print(f",,, {test1}")
            print(v)

------  dictionary_item_added
..... ITs an add statement
======= ['extensions', 'extension-definition--ef765651-680c-498d-9894-99799f2fa126', 'impact_refs']
['impact--dfb745f7-a931-4f3a-a6ef-6e9f8ae4a018']
------  dictionary_item_removed
..... ITs a remove statement
======= ['extensions', 'extension-definition--ef765651-680c-498d-9894-99799f2fa126', 'investigation_status']
new
------  values_changed
..... ITs an value changed statement
======= ['name']
{'new_value': 'Potential phishing', 'old_value': 'potential phishing'}
------  iterable_item_added
..... ITs an iterable item added statement
======= ['extensions', 'extension-definition--ef765651-680c-498d-9894-99799f2fa126', 'sequence_refs', 2]
sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be
======= ['extensions', 'extension-definition--ef765651-680c-498d-9894-99799f2fa126', 'task_refs', 1]
task--3d8b10a7-0217-4784-a17d-26f55a68da0a
======= ['extensions', 'extension-definition--ef765651-680c-498d-9894-99799f2fa126', 'other_object_refs',

In [8]:
sequence_start = DeepDiff(t_sequence_start_objs, sequence_start_objs, verbose_level=2)
print(sequence_start)

{}


In [9]:
sequences = DeepDiff(t_sequence_objs, sequence_objs, verbose_level=2)
print(sequences)

{'iterable_item_added': {'root[2]': {'type': 'sequence', 'spec_version': '2.1', 'id': 'sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be', 'created': '2023-12-31T08:03:29.010237Z', 'modified': '2023-12-31T08:03:29.010237Z', 'sequenced_object': 'task--3d8b10a7-0217-4784-a17d-26f55a68da0a', 'sequence_type': 'task', 'step_type': 'single_step', 'extensions': {'extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e': {'extension_type': 'new-sdo'}}, 'on_completion': 'sequence--27d43ed5-0a50-47da-9ca5-2df81f9873be'}}}


In [13]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}
C = A - B
print(f"Delete -> Things in old, not in new ->{C}")
D = B - A
print(f"Add -> Things in new, not in old ->{D}")
E = A & B
print(f"Intersections -> Things which may have changed ->{E}")


Delete -> Things in old, not in new ->{0, 8, 6}
Add -> Things in new, not in old ->{1, 3, 5}
Intersections -> Things which may have changed ->{2, 4}
